In [2]:
from model import *
import torch

sample_input = torch.randn(1, 300, 3, 480, 480)
sample_input = sample_input.permute(0, 2, 1, 3, 4)
classifier = CNN3D()
out = classifier(sample_input)
print(out.shape)

torch.Size([1, 1])


In [14]:
from dataLoader import *

labels_path = "../../dissData/labels/all_labels.json"
sample_vids = "../../dissData/allVids"

video_dataset = VideoDataset(sample_vids, labels_path, transform=None)

sample_index = 0
sample_frames, sample_label = video_dataset[sample_index]
print(sample_frames.shape)
print(sample_label)

torch.Size([300, 3, 480, 480])
1
